### 라이브러리

In [84]:
import urllib.request as urlreq
import os.path
from bs4 import BeautifulSoup as bs
import pandas as pd

### RSS 

In [116]:
url = "https://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
savename = 'forecast.xml'

In [118]:
if not os.path.exists(savename) :
    urlreq.urlretrieve(url, savename) 

xml = open(savename, encoding='utf-8').read() 
soup = bs(xml, 'lxml-xml') 
info = {}

for location in soup.find_all('location') : 
    name = location.find('city').string # 도시이름 
    weather = location.find('wf').string # 날씨
    tempMin = location.find('tmn').string # 최저기온
    tempMax = location.find('tmx').string # 최고기온
    
    if not name in info : 
        info[name] = [] 
    info[name].append(weather) 
    info[name].append(tempMin) 
    info[name].append(tempMax) 

df = pd.DataFrame(info) 
df.index = ['날씨', '최저기온', '최고기온'] # 인덱스 설정

print(df)

df.to_csv('weathers.csv', encoding='utf-8-sig')  # csv 변환

       서울   인천   수원   파주   이천   평택   춘천   원주  강릉   대전  ...  거창  통영  대구   안동  \
날씨     맑음   맑음   맑음   맑음   맑음   맑음   맑음   맑음  맑음   맑음  ...  맑음  맑음  맑음   맑음   
최저기온  -11  -10  -12  -16  -14  -12  -15  -13  -8  -10  ...  -9  -5  -8  -11   
최고기온   -1    0    0    0    0    0   -1   -1   2    1  ...   2   5   3    1   

      포항  경주  울진 울릉도     제주    서귀포  
날씨    맑음  맑음  맑음  맑음  흐리고 눈  흐리고 눈  
최저기온  -7  -9  -8  -4      3      2  
최고기온   3   3   4   0      6      8  

[3 rows x 41 columns]


### Oracle DB 연결

In [119]:
!pip install cx_Oracle
import cx_Oracle as xe
from bs4 import BeautifulSoup
import requests as req
from pprint import pprint
!pip install pymssql
import pyodbc

### Oracle DB : data 생성

In [120]:
dsn = cx_Oracle.makedsn('project-db-stu.ddns.net',1524,'xe')
url = "https://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
material = req.get(url)
soup = BeautifulSoup(material.text, 'html.parser')
conn = cx_Oracle.connect('cgi_8_1_1216','smhrd1',dsn)

with conn :
    cur = conn.cursor()
    drop_table = """
                    BEGIN
                        EXECUTE IMMEDIATE 'DROP TABLE forecast';
                    EXCEPTION
                        WHEN OTHERS THEN
                            IF SQLCODE != -942 THEN
                                RAISE;
                            END IF;
                    END;
                """
    cur.execute(drop_table)
    
    drop_seq = """
                    BEGIN
                        EXECUTE IMMEDIATE 'DROP SEQUENCE fc_seq';
                    EXCEPTION
                        WHEN OTHERS THEN
                        IF SQLCODE != -2289 THEN
                            RAISE;
                        END IF;
                    END;
                """
    cur.execute(drop_seq)
    
    create_table ="""
                    create table forecast(
                        city_index number(4) primary key,
                        city_name varchar2(20),
                        weather_stat varchar2(20),
                        temp_min number(5),
                        temp_max number(5)
                    )
                """
    cur.execute(create_table)
    
    cur.execute("create sequence fc_seq")
    
    for location in soup.find_all('location'):
        name = location.find('city').string
        weather = location.find('wf').string
        tempMin = int(location.find('tmn').string)
        tempMax = int(location.find('tmx').string)
        
        # 데이터 삽입
        insert_data = """
                        insert into forecast(city_index, city_name, weather_stat, temp_min, temp_max)
                        values(fc_seq.nextval, :1, :2, :3, :4)
                    """
        cur.execute(insert_data, (name, weather, tempMin, tempMax)) # 도시, 날씨, 최저기온, 최고기온
    conn.commit()

    cur.execute("select * from forecast") #데이터 조회 => 데이터 추가됐는지 확인
    pprint(cur.fetchall())

[(1, '서울', '맑음', -6, 3),
 (2, '인천', '맑음', -5, 3),
 (3, '수원', '맑음', -7, 4),
 (4, '파주', '맑음', -12, 2),
 (5, '이천', '맑음', -10, 3),
 (6, '평택', '맑음', -8, 4),
 (7, '춘천', '맑음', -10, 3),
 (8, '원주', '맑음', -8, 2),
 (9, '강릉', '맑음', -4, 7),
 (10, '대전', '맑음', -6, 5),
 (11, '세종', '맑음', -7, 5),
 (12, '홍성', '맑음', -7, 5),
 (13, '청주', '맑음', -6, 4),
 (14, '충주', '맑음', -10, 3),
 (15, '영동', '맑음', -9, 5),
 (16, '광주', '구름많음', -2, 7),
 (17, '목포', '구름많음', 1, 8),
 (18, '여수', '구름많음', 1, 8),
 (19, '순천', '구름많음', -1, 9),
 (20, '광양', '구름많음', -1, 9),
 (21, '나주', '구름많음', -4, 7),
 (22, '전주', '구름많음', -3, 6),
 (23, '군산', '구름많음', -5, 5),
 (24, '정읍', '구름많음', -3, 5),
 (25, '남원', '구름많음', -5, 6),
 (26, '고창', '구름많음', -2, 6),
 (27, '무주', '구름많음', -7, 5),
 (28, '부산', '맑음', 0, 8),
 (29, '울산', '맑음', -2, 8),
 (30, '창원', '맑음', -2, 7),
 (31, '진주', '맑음', -6, 8),
 (32, '거창', '맑음', -7, 7),
 (33, '통영', '맑음', 0, 8),
 (34, '대구', '맑음', -4, 7),
 (35, '안동', '맑음', -8, 5),
 (36, '포항', '맑음', -2, 8),
 (37, '경주', '맑음', -5, 8),
 (38, '울진', '맑음', -4, 8